In [13]:
import numpy as np

# Load the numpy arrays
python_boards = np.load('python_boards.npy')
typescript_boards = np.load('typescript_boards.npy')

# Ensure the arrays have the same shape
if python_boards.shape != typescript_boards.shape:
    raise ValueError("The arrays have different shapes and cannot be compared directly.")


In [14]:
def tensor_to_board(tensor: np.ndarray) -> str:
    piece_types = ["P", "N", "B", "R", "Q", "K", "p", "n", "b", "r", "q", "k"]
    board = [["" for _ in range(8)] for _ in range(8)]

    # Piece placement
    for i, piece in enumerate(piece_types):
        channel = tensor[i * 64 : (i + 1) * 64]
        for idx, val in enumerate(channel):
            if val == 1.0:
                row = idx // 8
                col = idx % 8
                board[row][col] = piece

    # Construct FEN piece placement
    fen_rows = []
    for row in board:
        fen_row = ""
        empty = 0
        for cell in row:
            if cell == "":
                empty += 1
            else:
                if empty > 0:
                    fen_row += str(empty)
                    empty = 0
                fen_row += cell
        if empty > 0:
            fen_row += str(empty)
        fen_rows.append(fen_row)
    piece_placement = "/".join(fen_rows)

    # Active color
    active_color_channel = tensor[12 * 64 : 13 * 64]
    active_color_sum = np.sum(active_color_channel).item()
    active_color = "w" if active_color_sum >= 32 else "b"

    # Castling rights
    castling_rights = ""
    castling_channels = tensor[13 * 64 : 17 * 64]
    rights = ["K", "Q", "k", "q"]
    for i in range(0, 256, 64):
        channel = castling_channels[i : i + 64]
        channel_sum = np.sum(channel).item()
        if channel_sum >= 32:
            castling_rights += rights[i // 64]
    castling_rights = castling_rights if castling_rights else "-"

    # En passant target
    ep_channel = tensor[17 * 64 : 18 * 64]
    ep_sum = np.sum(ep_channel).item()
    if ep_sum == 1.0:
        ep_index = np.argmax(ep_channel)
        file = ep_index % 8
        rank = 7 - (ep_index // 8)
        en_passant = f"{chr(ord('a') + file)}{rank + 1}"
    else:
        en_passant = "-"

    # Assemble FEN
    fen = (
        f"{piece_placement} {active_color} {castling_rights} {en_passant} 0 1"
    )
    return fen

In [15]:
# Convert tensors to FEN strings
python_board_fen = tensor_to_board(python_boards)
typescript_board_fen = tensor_to_board(typescript_boards)

print("Python Board FEN:")
print(python_board_fen)
print("\nTypeScript Board FEN:")
print(typescript_board_fen)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()